![](./img/NLP_banner.jpg)

# python中文新闻分类
#### \[稀牛学院 x 网易云课程\]《文本分类机器学习模型与实战》课程资料整理 by [@龙心尘](https://blog.csdn.net/longxinchen_ml)
* 参考代码：[【NLP文本分类】各种文本分类算法集锦，从入门到精通](https://www.kesci.com/home/project/5be7e948954d6e0010632ef2)
* 参考代码：[NLP系列(4)_朴素贝叶斯实战与进阶](https://blog.csdn.net/longxinchen_ml/article/details/50629613)

In [46]:
import re  
import pickle
import random
from sklearn.svm import SVC
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB



In [47]:
data = [[i.split("\t")[0],i.split("\t")[1]] for i in open(u"data/cnews.train.txt","r", encoding="utf-8")]
data[0]

['体育',
 '鲍勃库西奖归谁属？ NCAA最强控卫是坎巴还是弗神新浪体育讯如今，本赛季的NCAA进入到了末段，各项奖项的评选结果也即将出炉，其中评选最佳控卫的鲍勃-库西奖就将在下周最终四强战时公布，鲍勃-库西奖是由奈史密斯篮球名人堂提供，旨在奖励年度最佳大学控卫。最终获奖的球员也即将在以下几名热门人选中产生。〈〈〈\xa0NCAA疯狂三月专题主页上线，点击链接查看精彩内容吉梅尔-弗雷戴特，杨百翰大学“弗神”吉梅尔-弗雷戴特一直都备受关注，他不仅仅是一名射手，他会用“终结对手脚踝”一样的变向过掉面前的防守者，并且他可以用任意一支手完成得分，如果他被犯规了，可以提前把这两份划入他的帐下了，因为他是一名命中率高达90%的罚球手。弗雷戴特具有所有伟大控卫都具备的一点特质，他是一位赢家也是一位领导者。“他整个赛季至始至终的稳定领导着球队前进，这是无可比拟的。”杨百翰大学主教练戴夫-罗斯称赞道，“他的得分能力毋庸置疑，但是我认为他带领球队获胜的能力才是他最重要的控卫职责。我们在主场之外的比赛(客场或中立场)共取胜19场，他都表现的很棒。”弗雷戴特能否在NBA取得成功？当然，但是有很多专业人士比我们更有资格去做出这样的判断。“我喜爱他。”凯尔特人主教练多克-里弗斯说道，“他很棒，我看过ESPN的片段剪辑，从剪辑来看，他是个超级巨星，我认为他很成为一名优秀的NBA球员。”诺兰-史密斯，杜克大学当赛季初，球队宣布大一天才控卫凯瑞-厄尔文因脚趾的伤病缺席赛季大部分比赛后，诺兰-史密斯便开始接管球权，他在进攻端上足发条，在ACC联盟(杜克大学所在分区)的得分榜上名列前茅，但同时他在分区助攻榜上也占据头名，这在众强林立的ACC联盟前无古人。“我不认为全美有其他的球员能在凯瑞-厄尔文受伤后，如此好的接管球队，并且之前毫无准备。”杜克主教练迈克-沙舍夫斯基赞扬道，“他会将比赛带入自己的节奏，得分，组织，领导球队，无所不能。而且他现在是攻防俱佳，对持球人的防守很有提高。总之他拥有了辉煌的赛季。”坎巴-沃克，康涅狄格大学坎巴-沃克带领康涅狄格在赛季初的毛伊岛邀请赛一路力克密歇根州大和肯塔基等队夺冠，他场均30分4助攻得到最佳球员。在大东赛区锦标赛和全国锦标赛中，他场均27.1分，6.1个篮板，5.1次助攻，依旧如此给力。他以疯狂的表现开始这个赛季，也将以疯狂的表现结束这个赛季。“我们在全国锦标

In [48]:
random.shuffle(data)

In [88]:
import jieba
jieba.enable_parallel() #并行分词开启
terms = [" ".join([j for j in jieba.cut(i[1])]) 
         for i in data[:1000]]


In [89]:
#接下来用scikit-learn中的LabelEncoder将文本标签（Text Label）转化为数字(Integer)

lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform([i[0] for i in data[:1000]])


In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer as TFIV
# 初始化TFIV对象，去停用词，加2元语言模型
tfv = TFIV(min_df=3,  max_features=None, strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}', ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1, stop_words = 'english')


# 这一步有点慢，去喝杯茶刷会儿微博知乎歇会儿...
tfv.fit(terms)
X_all = tfv.transform(terms)



In [91]:
len_train = 600
X = X_all[:len_train] 
y_train = y[:len_train]
X_test = X_all[len_train:]
y_test = y[len_train:]


In [92]:
# 多项式朴素贝叶斯
from sklearn.naive_bayes import MultinomialNB as MNB

# 因为sklearn的NB只支持二分类，所以就简单粗暴地把10个类变成一个类。
y_train_nb = [1 if i > 5 else 0 for i in y_train]
model_NB = MNB()
model_NB.fit(X, y_train_nb ) #特征数据直接灌进来
MNB(alpha=1.0, class_prior=None, fit_prior=True) # ”alpha“是平滑参数，不需要掌握哈。

from sklearn.cross_validation import cross_val_score
import numpy as np

print("多项式贝叶斯分类器20折交叉验证得分: ", np.mean(cross_val_score(model_NB, X, y_train_nb, cv=20, scoring='roc_auc')))


多项式贝叶斯分类器20折交叉验证得分:  0.9858054226475279


In [93]:
# 折腾一下逻辑回归，恩
from sklearn.linear_model import LogisticRegression as LR
from sklearn.grid_search import GridSearchCV

model_LR = LogisticRegression(C=.01) # C是正则化系数。
model_LR.fit(X, y_train_nb)
print("20折交叉验证得分: ", np.mean(cross_val_score(model_LR, X, y_train_nb, cv=20, scoring='roc_auc')))


20折交叉验证得分:  0.9685871876661348


In [94]:
from sklearn.svm import LinearSVC
model_SVM = LinearSVC(C=.01) # C是正则化系数。
model_SVM.fit(X, y_train_nb)
print("20折交叉验证得分: ", np.mean(cross_val_score(model_SVM, X, y_train_nb, cv=20, scoring='roc_auc')))

20折交叉验证得分:  0.9811038011695905


In [100]:
model_LR = LogisticRegression(C=.01) # C是正则化系数。
model_LR.fit(X, y_train)
print("20折交叉验证得分: ", np.mean(cross_val_score(model_LR, X, y_train, cv=20)))


20折交叉验证得分:  0.27262206942163847


In [99]:
model_SVM = LinearSVC(C=.01) # C是正则化系数。
model_SVM.fit(X, y_train)
print("20折交叉验证得分: ", np.mean(cross_val_score(model_SVM, X, y_train, cv=20)))

20折交叉验证得分:  0.8104821053375002


## 版权归 © 稀牛学院 所有 保留所有权利
![](./img/xiniu_neteasy.png)